<a href="https://colab.research.google.com/github/25maikelo/TI_1_Practica3/blob/main/TI_1_Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Miguel Ángel Lozano López
**e-mail:** miguel.lozano9074@alumnos.udg.mx

## MODULES

In [1]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [11]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vector_ab = [pos_b, pos_a]
    norm_ab = np.linalg.norm(vector_ab) # Norma / Magnitud de vector AB

    vector_bc = [pos_c, pos_b]
    norm_bc = np.linalg.norm(vector_bc) # Norma / Magnitud de vector AB

    dot_p = np.dot(vector_ab, vector_bc) # Producto punto

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = (dot_p) / np.finfo((norm_ab * norm_bc) ).eps

    # Angle orientation
    cross_p = np.cross(vector_ab, vector_bc)

    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [12]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [16]:
# Define your function to compute path length for given trajectory
def path_length(trajectory):

    distance_df = np.array([distance.euclidean(trajectory.iloc[i-1], trajectory.iloc[i]) for i in range(1, trajectory.shape[0])])
    # path_length_df = np.cumsum(distance_df)
    path_length_df = np.array([sum(distance_df[:i+1]) for i in range(len(distance_df))])
    return path_length_df

In [17]:
# Get Path length calling the function
PL_BM_3 = path_length(BM_2d_df_3)

# Get Path length calling the function
PL_BM_6 = path_length(BM_2d_df_6)

# Get Path length calling the function
PL_CRW_6 = path_length(CRW_2d_df_9)

In [18]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# Add BM3
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_BM_3)) + 1,
    y=PL_BM_3,
    name='Path Length BM 3',
    showlegend=True
))

# Add BM6
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_BM_6)) + 1,
    y=PL_BM_6,
    line=dict(width=6),
    name='Path Length BM 6',
    showlegend=True
))

# Add CRW6
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_CRW_6)) + 1,
    y=PL_CRW_6,
    name='Path Length CRW 6',
    showlegend=True
))


fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [19]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [ ]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [318]:
# Define your function to compute Mean Squared Displacement for given trajectory
def get_mean_squared_displacement(trajectory):

    N = len(trajectory)
    msd_list = []

    for n in range(1, N):

      squared_distances = []
      for i in range(N - n):

        displacement = distance.euclidean(trajectory.iloc[i], trajectory.iloc[i + n])
        squared_distances.append(pow(displacement, 2))

      msd_list.append(np.sum(squared_distances) / (N - n))

    result = msd_list
    return result


In [319]:
# Get Mean Squared Displacement calling the function
MSD_BM = get_mean_squared_displacement(BM_2d_df_6)

# Get Mean Squared Displacement calling the function
MSD_CRW = get_mean_squared_displacement(CRW_2d_df_9)

In [324]:
# Init figure
fig_MSD = go.Figure()

# first trace MSD_BM
fig_MSD.add_trace(go.Scatter(
    x=BM_2d_df_6.index,
    y=MSD_BM,
    name='MSD BM 6',
    showlegend=True
))

# Second trace MSD_CRW
fig_MSD.add_trace(go.Scatter(
    x=CRW_2d_df_9.index,
    y=MSD_CRW,
    name='MSD CRW 6 c=0.9',
    showlegend=True
))


fig_MSD.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [158]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [159]:
# Define your function to compute Turning Angles for given trajectory
def get_turning_angles(trajectory):
    """
    Arguments:
        trajectory: Full trajectory
    Returns:
        turning_angles_df: Turning angles from trajectories
    """

    turning_angles = []
    for i in range(1, len(trajectory) - 1):

      # Positions
      pos_a = trajectory.iloc[i - 1]  # Row i - 1
      pos_b = trajectory.iloc[i]      # Row i
      pos_c = trajectory.iloc[i + 1]  # Row i + 1

      # Vectors
      vector_ab = pos_b - pos_a
      vector_bc = pos_c - pos_b

      # Norm / AB Vector Magnitude
      norm_ab = np.linalg.norm(vector_ab)
      norm_bc = np.linalg.norm(vector_bc)

      dot_p = np.dot(vector_ab, vector_bc) # Dot product
      cos_theta = dot_p / (norm_ab * norm_bc)

      # Angle orientation
      cross_p = np.cross(vector_ab, vector_bc)

      orientation = np.sign(cross_p)

      if orientation == 0:
          orientation = 1

      # Get angle in radians
      theta = np.arccos(np.around(cos_theta, 4)) * orientation
      turning_angles.append(theta)

    turning_angles_df = pd.DataFrame(turning_angles, columns=['angles'])
    return turning_angles_df

In [160]:
# Get Turning Angles calling the function
ta_CRW_2d_df_6 = get_turning_angles(CRW_2d_df_6)

# Get Turning Angles calling the function
ta_CRW_2d_df_9 = get_turning_angles(CRW_2d_df_9)

In [161]:
# Check documentation
# https://plotly.com/python/histograms/

# Plot histogram
fig_turning_angles_dist = go.Figure()


# Histogram turning angle CRW_2d_df_6
histogram_6 = np.array([wrapcauchy.pdf(x=i, c=0.6) if i >= 0 else wrapcauchy.pdf(x=-i, c=0.6) for i in ta_CRW_2d_df_6.angles])
histograma_crw_2d_6 = go.Histogram(x=histogram_6, name='Observed 0.6', nbinsx=100, histnorm='probability density')

# Histogram turning angle CRW_2d_df_9
histogram_9 = np.array([wrapcauchy.pdf(x=i, c=0.9) if i >= 0 else wrapcauchy.pdf(x=-i, c=0.9) for i in ta_CRW_2d_df_9.angles])
histograma_crw_2d_9 = go.Histogram(x=histogram_9, name='Observed 0.9', nbinsx=100, histnorm='probability density')

# Add both histograms
fig_turning_angles_dist.add_trace(histograma_crw_2d_6)
fig_turning_angles_dist.add_trace(histograma_crw_2d_9)


# Add origin distributions
resolution = 1000
aux_domain = np.linspace(-np.pi, np.pi, resolution)
wrapcauchy_pdf_6 = np.array([wrapcauchy.pdf(x=i, c=0.6) if i >= 0 else wrapcauchy.pdf(x=-i, c=0.6) for i in aux_domain])
wrapcauchy_pdf_9 = np.array([wrapcauchy.pdf(x=i, c=0.9) if i >= 0 else wrapcauchy.pdf(x=-i, c=0.9) for i in aux_domain])

# Plot
fig_turning_angles_dist.add_trace(
  go.Scatter(
    x=aux_domain,
    y=wrapcauchy_pdf_6,
    marker=dict(size=2),
    mode='lines',
    name='Cauchy 0.6',
    showlegend=True
  )
)

fig_turning_angles_dist.add_trace(
  go.Scatter(
    x=aux_domain,
    y=wrapcauchy_pdf_9,
    marker=dict(size=2),
    mode='lines',
    name='Cauchy 0.9',
    showlegend=True
  )
)


fig_turning_angles_dist.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [ ]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
# def function (trajectory):


    #return sl_df

## end - Add your code here

In [ ]:
# Get Step lengths calling the function
sl_Levy_2d_df_1 = # Add your code here function(Levy_2d_df_1)

# Get Step lengths calling the function
sl_Levy_2d_df_7 = # Add your code here function(Levy_2d_df_7)

In [ ]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here

## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here

## end - Add your code here


# Add origin distributions
## start - Add your code here

## end - Add your code here


fig_met_df_4.show()